In [6]:
from notebook_autorun import Autorun
Autorun.info()
cells = '1:6'
Autorun(cells=cells).add_js()

### notebook-autorun

This module enables auto-run of certains cells at notebook kernel start.  
It will work **only** for <span style="color: red">**trusted**</span> notebooks.  

#### 1 - Settings

The cells to autorun can be determined by one of the 3 args below.  
Only one of them must be specified:  
+ `cells`: List of 0-indexed cells or String representing a list of cells. [Python list slices](https://docs.python.org/2.3/whatsnew/section-slices.html) are allowed. Default is `None`. `cells` is stringified to `str_cells` before passing to javascript. Invalid cell numbers (e.g. greater than the notebook number of cells) are ignored. Examples: [2, 3], '2,3', '2:', ':-2', '4:8', '::-1'  
+ `metadata`: Boolean (default is `False`). If `True`, all cells with medatada `"autorun": true` are concerned.  
+ `comment`: Boolean (default is `False`). If `True`, all cells containing a comment `comment_flag` (default is `# AUTORUN`) are concerned.  


#### 2 - Security

Because a notebook is designed to allow the user to write arbitrary code, it has full access to many resources.   

The typical risks are the following:
+ A notebook has access to your file system and can therefore potentially read/modify/delete any of your files or send them to an attacker, or write a new file (virus).  
+ A notebook may contain javascript in output cells which can read you cookies and local storage and potentially send them to an attacker.  

See the [Security in notebook documents](https://jupyter-notebook.readthedocs.io/en/stable/security.html#security-in-notebook-documents) section of the official [Jupyter Notebook documentation](https://jupyter-notebook.readthedocs.io/en/stable/index.html) for more info.  

Therefore you **should review** and **must trust** the notebook before you can use **notebook-autorun**.

If you see <IPython.core.display.Javascript object> below, this notebook is not trusted.
As a consequence Autorun cannot work.
Run "from notebook_autorun import Autorun; Autorun.info()" for more info.


<IPython.core.display.Javascript object>

This output cell contains **notebook-autorun** settings:  
```json
   {"str_cells": "1:6"}  
```

In [1]:
#import os
#os.environ["SIMCORE_CONFIG_PATH"] = r"C:\Users\anderegg\Desktop\alternative_config.json"

import simcoreapi
from simcoreapi import PORTS
from simcoreapi import config

data_path = PORTS.inputs[0].value

In [ ]:
import plotly.offline as offline
import plotly.figure_factory as ff
import plotly.graph_objs as go
import pandas as pd
import os

offline.init_notebook_mode(connected=True)

column_name_map = {0: {"name":"time [s]"}, 1: {"name":"rate"}, 2: {"name":"demand"}, 
                   3: {"name":"move/(max - min) sympathetic efferent", "scale":1/0.0008}, 
                   4: {"name":"threshold indirect parasympathetic efferent", "scale":-1.0},
                   5: {"name":"direct parasympathetic efferent", "scale":-1/0.0008}, 
                   6: {"name":"recruitment = coefficient of variation"}}


data_frame = pd.read_csv(data_path, sep=',', names=[column_name_map[i]["name"] for i in column_name_map.keys()])

trace = go.Table(
    header=dict(
        values=data_frame.columns,
        fill=dict(color="#C2D4FF"),
        align=["left"]
    ),
    cells=dict(
        values=[data_frame[data_frame.columns[i]] for i in column_name_map.keys()],
        fill=dict(color="#F5F8FF"),
        align=["left"]
    )
)
data = [trace]

offline.iplot(data, filename='data-table', config={"displayModeBar": False})

In [3]:
def create_graph(data_frame, data_scaling, title, x_axis_title):
    data = [
        go.Scatter(
            x=data_frame[data_frame.columns[0]],
            y=data_frame[data_frame.columns[i]] * data_scaling[i], 
            opacity=0.5,                        
            name=data_frame.columns[i]
        ) for i in range(1,data_frame.columns.size)
    ]

    layout = go.Layout(
        title=title, 
        showlegend=True,
        xaxis=dict(
            title=x_axis_title
        ),
        yaxis=dict(
            #range=[0,.5]    
        ))
    fig = go.Figure(data=data, layout=layout)
    offline.iplot(fig, config={"displayModeBar": False})

In [ ]:
title="Heart Rate(r)"
heart_axis_colums = [0,1,2,3,5]
heart_rate_data = data_frame.filter(items=[data_frame.columns[i] for i in heart_axis_colums])
heart_rate_data_scaling = [column_name_map[i]["scale"] if "scale" in column_name_map[i] else 1 for i in heart_axis_colums]
create_graph(data_frame=heart_rate_data, 
             data_scaling=heart_rate_data_scaling, 
             title=title, 
             x_axis_title="time(sec)")

In [ ]:
import pprint
print("There are ", len(PORTS.inputs), " inputs")
for _in in PORTS.inputs:
    pprint.pprint(_in._asdict)



In [ ]:
print("There are ", len(simcore.outputs), " outputs")
for _out in simcore.outputs:
    pprint.pprint(_out._asdict)
